In [ ]:
from scipy.io import loadmat
import albumentations as A
import torch
import torchvision.models as models
import cv2
from albumentations.pytorch import ToTensorV2
from torch import nn
import math
import glob
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
import pandas as pd
from torch.nn.parameter import Parameter

In [ ]:
class CFG:
    model_name = 'efficienet_b3'
    path_test = "../input/qmulsurvfacev1/QMUL-SurvFace/Face_Verification_Test_Set/verification_images"
    verification_neg_pairs = "./negative_pairs_names.mat"
    verification_pos_pairs = "./pos_pairs_names.mat"
    pretrain_model = "../input/effb3-arcface-no-agu-30epoch/best_model (38).pt"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_classes = 5319
    input_size = 112
    use_pretrained = True
    path_train = "../input/train-valid-qmul/data/train/"
    ### model config
    use_pretrained = True
    model_name = "efficientnet_b3"
    num_classes = len(glob.glob(path_train+"*"))
    embedding_size = 512
    train = True
    
    dropout = 0.3
    metric = 'arcface' # arcface, cosface , softmax 
    use_fc = True
    s = 30
    margin = 0.5
    ls_eps = 0.0
    theta_zero = 0.785

In [ ]:
class AdaCos(nn.Module):
    def __init__(self, in_features, out_features, m=0.50, ls_eps=0, theta_zero=math.pi/4):
        super(AdaCos, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.theta_zero = theta_zero
        self.s = math.log(out_features - 1) / math.cos(theta_zero)
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        # normalize features
        x = F.normalize(input)
        # normalize weights
        W = F.normalize(self.weight)
        # dot product
        logits = F.linear(x, W)
        # add margin
        theta = torch.acos(torch.clamp(logits, -1.0 + 1e-7, 1.0 - 1e-7))
        target_logits = torch.cos(theta + self.m)
        one_hot = torch.zeros_like(logits)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        output = logits * (1 - one_hot) + target_logits * one_hot
        # feature re-scale
        with torch.no_grad():
            B_avg = torch.where(one_hot < 1, torch.exp(self.s * logits), torch.zeros_like(logits))
            B_avg = torch.sum(B_avg) / input.size(0)
            theta_med = torch.median(theta)
            self.s = torch.log(B_avg) / torch.cos(torch.min(self.theta_zero * torch.ones_like(theta_med), theta_med))
        output *= self.s

        return output
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output
class AddMarginProduct(nn.Module):
    r"""Implement of large margin cosine distance: :
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        s: norm of input feature
        m: margin
        cos(theta) - m
    """

    def __init__(self, in_features, out_features, s=30.0, m=0.40):
        super(AddMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        phi = cosine - self.m
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device=CFG.device)
        # one_hot = one_hot.cuda() if cosine.is_cuda else one_hot
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output

class LiArcFace(nn.Module):
    def __init__(self, in_features, out_features, m=0.4, s=64.0):
        super().__init__()
        self.weight = nn.Parameter(torch.empty(out_features, in_features))
        nn.init.xavier_normal_(self.weight)
        self.m = m
        self.s = s

    def forward(self, input, label):
        W = F.normalize(self.weight)
        input = F.normalize(input)
        cosine = input @ W.t()
        theta = torch.acos(cosine)
        m = torch.zeros_like(theta)
        m.scatter_(1, label.view(-1, 1), self.m)
        scale = -2 * self.s / math.pi
        return self.s + scale * (theta + m)

In [ ]:
class Model(nn.Module):

    def __init__(self,
                 n_classes,
                 model_name='efficientnet_b3',
                 use_fc=False,
                 fc_dim=512,
                 dropout=0.0,
                 metric='softmax',
                 s=30.0,
                 margin=0.50,
                 ls_eps=0.0,
                 theta_zero=0.785,
                 pretrained=False):
        """
        :param n_classes:
        :param model_name: name of model from pretrainedmodels
            e.g. resnet50, resnext101_32x4d, pnasnet5large
        :param pooling: One of ('SPoC', 'MAC', 'RMAC', 'GeM', 'Rpool', 'Flatten', 'CompactBilinearPooling')
        :param metric: One of ('arcface', 'cosface', 'softmax')
        """
        super(Model, self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=True)
#         self.backbone.load_state_dict(torch.load(CFG.orginal_pretrain_model, map_location=CFG.device))
        final_in_features = self.backbone.classifier.in_features
        
        self.backbone.classifier = nn.Identity()
        self.backbone.global_pool = nn.Identity()
        
        self.pooling =  nn.AdaptiveAvgPool2d(1)
            
        self.use_fc = use_fc
        if use_fc:
            self.dropout = nn.Dropout(p=dropout)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            final_in_features = fc_dim

        self.metric = metric
        if metric == 'arcface':
            self.final = ArcMarginProduct(final_in_features, n_classes,
                                          s=s, m=margin, easy_margin=False, ls_eps=ls_eps)
        elif metric == 'cosface':
            self.final = AddMarginProduct(final_in_features, n_classes, s=s, m=margin)
        elif metric == 'adacos':
            self.final = AdaCos(final_in_features, n_classes, m=margin, theta_zero=theta_zero)
        elif metric == 'LiArcFace':
            print("LiArcFace")
            self.final = LiArcFace(final_in_features, n_classes)
        else:
            self.final = nn.Linear(final_in_features, n_classes)

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, x, label):
        feature = self.extract_feat(x)
        if self.metric in ('arcface', 'cosface', 'adacos','LiArcFace'):
            logits = self.final(feature, label)
        else:
            logits = self.final(feature)
        return logits

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)

        return x

In [ ]:
class FocalLoss(torch.nn.Module):

    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [ ]:
!cp "../input/qmulsurvfacev1/QMUL-SurvFace/Face_Verification_Test_Set/negative_pairs_names.mat" "negative_pairs_names.mat"
!cp "../input/qmulsurvfacev1/QMUL-SurvFace/Face_Verification_Test_Set/positive_pairs_names.mat" "positive_pairs_names.mat"
neg_pairs = loadmat("./negative_pairs_names.mat")['negative_pairs_names']
pos_pairs = loadmat("./positive_pairs_names.mat")['positive_pairs_names']

In [ ]:
# !rm "./neg_pairs.csv"
# !rm "./pos_pairs.csv"

In [ ]:
model = Model(**{
            'n_classes':CFG.num_classes,
            'model_name':CFG.model_name,
            'use_fc':CFG.use_fc,
            'fc_dim':CFG.embedding_size,
            'dropout':CFG.dropout,
            'metric':CFG.metric,
            's':CFG.s,
            'margin':CFG.margin,
            'ls_eps':CFG.ls_eps,
            'theta_zero':CFG.theta_zero,
            'pretrained':True
        }
)
if CFG.pretrain_model is not None:
    print("load pretrain")
    checkpoint = torch.load(CFG.pretrain_model,map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])    
    
trans = A.Compose([
        A.Resize(CFG.input_size, CFG.input_size),
        A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        A.pytorch.transforms.ToTensorV2()
])

model.to(CFG.device)
model.eval()
for name in ['neg_pairs','pos_pairs']:
    print("phase: ", name)
    time = 0
    sum_ = 0
    result = []
    for pairs in eval(name):
        img1= CFG.path_test+"/"+pairs[0][0]
        img2= CFG.path_test+"/"+pairs[1][0]

        sample1 = cv2.imread(img1)
        sample1= cv2.cvtColor(sample1, cv2.COLOR_BGR2RGB)
        sample2 = cv2.imread(img2)
        sample2= cv2.cvtColor(sample2, cv2.COLOR_BGR2RGB)

        sample1 = trans(image=sample1)['image'].to(CFG.device)
        sample2 = trans(image=sample2)['image'].to(CFG.device)

        embeding1 = model.extract_feat(sample1[None,...])
        embeding2 = model.extract_feat(sample2[None,...])

        score = torch.nn.functional.cosine_similarity(embeding1, embeding2).detach().cpu().numpy()[0]
        print(score)
        result.append(score)
        time+=1
    pd.DataFrame({'result':result}).to_csv(name+".csv")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
initial_threshold = 1.1
range_ = 0.9
step = 0.001
thresholds = []

In [ ]:
neg = pd.read_csv("neg_pairs.csv")
pos = pd.read_csv("pos_pairs.csv")

In [ ]:
def calculate_accuracy(list_positive_scores, list_negative_scores):
    temp_accuracy = 0
    initial_threshold = 1.1;
    range = 0.9;
    step = 0.001;
    optimial_threshold = 1.1
    optimal_threshold = 0
    thresholds = np.arange(initial_threshold-range_, initial_threshold+range_, step=0.001)
    for threshold in thresholds:
        current_accuracy_pos = [score for score in list_positive_scores if score > threshold]
        current_accuracy_neg = [score for score in list_negative_scores if score < threshold]
        current_accuracy = (len(current_accuracy_neg) + len(current_accuracy_pos))/(len(list_negative_scores) + len(list_positive_scores))
        if current_accuracy > temp_accuracy:
            temp_accuracy = current_accuracy
            optimal_threshold = threshold
    return optimal_threshold, temp_accuracy

In [ ]:
optimal_threshold, temp_accuracy = calculate_accuracy(pos['result'].values.tolist(),neg['result'].values.tolist())
print(optimal_threshold, temp_accuracy)

In [ ]:
pos['result'].values.tolist()

In [ ]:
list_negative_scores = neg['result'].values
list_positive_scores = pos['result'].values
thresholds = []
initial_threshold=1.1
range_ = 0.9
FARs = []
TARs = []
for threshold in np.arange(initial_threshold-range_, initial_threshold+range_, step=0.001):
    current_far = len([score for score in list_negative_scores if score > threshold])/len(list_negative_scores)
    current_tar = len([score for score in list_positive_scores if score < threshold])/len(list_positive_scores)
    thresholds.append(threshold)
    FARs.append(current_far)
    TARs.append(current_tar)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(thresholds, FARs)
plt.xlabel('thresholds')
plt.ylabel('FARs')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(thresholds, TARs)
plt.xlabel('thresholds')
plt.ylabel('FARs')
plt.show()

In [ ]:
FARs = []
thresh_holds = np.arange(initial_threshold - range_, initial_threshold + range_, step)
for threshold in thresh_holds:
    current_far = (np.sum(neg.values<threshold))/(neg_pairs.shape[0])
    FARs.append(current_far)
TARs = []
for threshold in thresholds:
    current_tar = (np.sum(pos.values<threshold))/pos_pairs.shape[0]
    TARs.append(current_tar)
